# 一.BeautifulSoup4将复杂的HTML文档转换成一个复杂的树形结构，每个节点都是python对象，所有对象可以归纳为4种：
- Tag
- NavigableString
- BeautifulSoup
- Comment


## 第一种：标签Tag

In [8]:
import re

from bs4 import BeautifulSoup
from select import select

#模拟从网页上爬取到了网页源代码
file=open("./sample.html","rb")
html=file.read().decode("utf-8")

#使用bs4需要指定解析器，html的解析器是“html.parser”
bs=BeautifulSoup(html,"html.parser")

#使用bs对象可以访问html里的一些标签，类似于树型查找
#下面这种返回的是第一个找到的标签
print(bs.title)
print(bs.a)
print(bs.head)
#这种就是bs对象的第一种元素，标签，这种只能拿到第一个标签
print(type(bs.head))

<title>练习网页</title>
<a class="link" href="https://www.example.com">点击这里访问示例网站</a>
<head>
<meta charset="utf-8"/>
<title>练习网页</title>
</head>
<class 'bs4.element.Tag'>


## 第二种：标签里的字符串NavigableString

In [7]:
print(bs.title.string)
print(type(bs.title.string))
#拿到一个标签里所有属性的方法
print(bs.a.attrs)

练习网页
<class 'bs4.element.NavigableString'>
{'href': 'https://www.example.com', 'class': ['link']}


## 第三种：网页源代码文档BeautifulSoup

In [9]:
print(bs)
print(type(bs))

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>练习网页</title>
</head>
<body>
<h1 id="main-title">欢迎学习BeautifulSoup</h1>
<p class="description">这是一个简单的HTML页面，用于学习Python爬虫。</p>
<div class="content">
<ul id="item-list">
<li class="item">苹果</li>
<li class="item">香蕉</li>
<li class="item">橘子</li>
</ul>
</div>
<a class="link" href="https://www.example.com">点击这里访问示例网站</a>
<a class="link" href="https://www.python.org">Python官网</a>
</body>
</html>

<class 'bs4.BeautifulSoup'>


## 第四种：特殊的第二种，但是忽略注释，不常用


# 二.如何应用？

## 1.文档的遍历

In [10]:
#可以获取这个标签内的另外的所有标签，并且放到一个列表中
print(bs.head.contents)
#因为返回值是一个列表，可以用中括号访问
print(bs.head.contents[1])

['\n', <meta charset="utf-8"/>, '\n', <title>练习网页</title>, '\n']
<meta charset="utf-8"/>


## 2.文档的搜索

In [33]:
#（1）find_all()

#第一种，使用字符串过滤：会查找和字符串完全相同的标签
t_list=bs.find_all("a")
print(t_list)
print("-----------------------------------")
#第二种：正则表达式搜索
import re
#包含a的都会被搜索出来
print(bs.find_all(re.compile("a")))
print("-----------------------------------")
#第三种：传入自定义函数进行搜索
def name_is_exist(tag):
    return tag.has_attr("id")
print(bs.find_all(name_is_exist))#为什么函数传入不需要参数？
#因为find_all在内部会遍历 HTML 中所有标签，然后逐个将标签对象作为参数传入你提供的函数
print("-----------------------------------")
#第四种，按照特点传入参数
for item in bs.find_all(id="main-title"):
    print(item)
for item in bs.find_all(class_="item"):#这里的class_是避免和关键字冲突
    print(item)
print("-----------------------------------")
#第五种，使用string（text在新版已被代替）按照文本查找（所以这个按字符串查找有区别？
for item in bs.find_all(string="苹果"):
    print(item)
#查找有数字的文本有哪些
for item in bs.find_all(string=re.compile("练习")):
    print(item)
print("-----------------第六种------------------")
#第六种，limit限制找的数量
print(bs.find_all("a",limit=1))

#补充，似乎各种方式可以在一个find_all（）里用逗号一块用

[<a class="link" href="https://www.example.com">点击这里访问示例网站</a>, <a class="link" href="https://www.python.org">Python官网</a>]
-----------------------------------
[<head>
<meta charset="utf-8"/>
<title>练习网页</title>
</head>, <meta charset="utf-8"/>, <a class="link" href="https://www.example.com">点击这里访问示例网站</a>, <a class="link" href="https://www.python.org">Python官网</a>]
-----------------------------------
[<h1 id="main-title">欢迎学习BeautifulSoup</h1>, <ul id="item-list">
<li class="item">苹果</li>
<li class="item">香蕉</li>
<li class="item">橘子</li>
</ul>]
-----------------------------------
<h1 id="main-title">欢迎学习BeautifulSoup</h1>
<li class="item">苹果</li>
<li class="item">香蕉</li>
<li class="item">橘子</li>
-----------------------------------
苹果
练习网页
-----------------第六种------------------
[<a class="link" href="https://www.example.com">点击这里访问示例网站</a>]


In [43]:
# （2）CSS选择器
print(bs.select("title"))
print(bs.select(".link"))#.表示后面是类名
print(bs.select("#item-list"))# #表示id
print(bs.select("p[class='description']"))# []表示属性
print(bs.select("head > title"))#通过层级(子标签 > ;兄弟标签 ~)查找

[<title>练习网页</title>]
[<a class="link" href="https://www.example.com">点击这里访问示例网站</a>, <a class="link" href="https://www.python.org">Python官网</a>]
[<ul id="item-list">
<li class="item">苹果</li>
<li class="item">香蕉</li>
<li class="item">橘子</li>
</ul>]
[<p class="description">这是一个简单的HTML页面，用于学习Python爬虫。</p>]
[<title>练习网页</title>]


# 三.正则表达式的学习

## 1.正则表达式常见操作符

## 2.re库的主要函数

In [54]:
import re

#创建模式对象
pat=re.compile("AA")#参数是正则表达式
print(pat.search("AAAAB"))#参数是待匹配的内容,返回的span是左闭右开，只会找到第一个AA

#不用模式对象也可以,前面是规则，后面是被校验串
print("---------------------------------------------------------")
print(re.search("asd","Aasd"))
#find_all()可以找到所有符合的
print("---------------------------------------------------------")
print(re.findall("asd","Aasdasd"))
print(re.findall("[A-Z]","AasdaBdasd"))
print(re.findall("[A-Z]+","ABCasdasABHdasd"))
#sub()找到1参用2参替换，在3参中
print("---------------------------------------------------------")
print(re.sub("a","A","aaaASDF"))#应用场景是把换行符换掉之类的
#注：建议在别比较的字符串前加入r一面转义字符的影响

<re.Match object; span=(0, 2), match='AA'>
---------------------------------------------------------
<re.Match object; span=(1, 4), match='asd'>
---------------------------------------------------------
['asd', 'asd']
['A', 'B']
['ABC', 'ABH']
---------------------------------------------------------
AAAASDF
